In [3]:
include("../src/AlphaStructures.jl") # nuovo modulo AlphaStructures
using LinearAlgebraicRepresentation, ViewerGL
using TimerOutputs
Lar = LinearAlgebraicRepresentation
GL =  ViewerGL
using BenchmarkTools


"""
    pointsRand(V, VV, n, m)

Generate random points inside and otuside `(V, VV)`.
"""
function pointsRand(
        V::Lar.Points, EV::Lar.Cells, n = 1000, m = 0
    )::Tuple{Lar.Points, Lar.Points, Lar.Cells, Lar.Cells}
    classify = Lar.pointInPolygonClassification(V, EV)
    Vi = [0;0]
    Ve = [0;0]
    k1 = 0
    k2 = 0
    while k1 < n || k2 < m
        queryPoint = [rand();rand()]
        inOut = classify(queryPoint)

        if k1 < n && inOut == "p_in"
            Vi = hcat(Vi, queryPoint)
            k1 = k1 + 1;
        end
        if k2 < m && inOut == "p_out"
            Ve = hcat(Ve, queryPoint)
            k2 = k2 + 1;
        end
    end
    VVi = [[i] for i = 1 : n]
    VVe = [[i] for i = 1 : m]
    return Vi[:,2:end], Ve[:,2:end], VVi, VVe
end

filename = "../examples/examples2D/svg_files/Lar2.svg";

V,EV = Lar.svg2lar(filename);

Vi, Ve, VVi, VVe = pointsRand(V, EV, 1000, 10000);

GL.VIEW([
    GL.GLGrid(Vi, VVi, GL.COLORS[1], 1)
    GL.GLGrid(Ve, VVe, GL.COLORS[12], 1)
])

filtration = AlphaStructures.alphaFilter(Vi);

VV,EV,FV = AlphaStructures.alphaSimplex(Vi, filtration, 0.02)

points = [[p] for p in VV]
faces = [[f] for f in FV]
edges = [[e] for e in EV]
GL.VIEW(
    GL.GLExplode(Vi, [edges; faces], 1.5, 1.5, 1.5, 99, 1)
 );

filter_key = sort(unique(values(filtration)))

granular = 10

reduced_filter = filter_key[sort(abs.(rand(Int, granular).%length(filter_key)))]
reduced_filter = [reduced_filter; max(filter_key...)]

#
# Arlecchino's Lar
#

for α in reduced_filter
    @show α
    VV,EV,FV = AlphaStructures.alphaSimplex(Vi, filtration, α)
    GL.VIEW(
        GL.GLExplode(
            Vi,
            [[[f] for f in FV]; [[e] for e in EV]],
            1., 1., 1.,	# Explode Ratio
            99, 1		# Colors
        )
    )
end

#
# Appearing Colors
#

reduced_filter = [
    0.002;	0.003;	0.004;	0.005;  0.006;
    0.007;	0.008;	0.009;	0.010;	0.013;
    0.015;	0.020;	0.050;	1.000
]

for i = 2 : length(reduced_filter)
    VV0, EV0, FV0 = AlphaStructures.alphaSimplex(Vi, filtration, reduced_filter[i-1])
    VV,  EV,  FV  = AlphaStructures.alphaSimplex(Vi, filtration, reduced_filter[i])
    EV0mesh = GL.GLGrid(Vi, EV0)
    FV0mesh = GL.GLGrid(Vi, FV0)
    EVmesh = GL.GLGrid(Vi, setdiff(EV, EV0), GL.COLORS[2], 1)
    FVmesh = GL.GLGrid(Vi, setdiff(FV, FV0), GL.COLORS[7], 1)
    GL.VIEW([EV0mesh; FV0mesh; EVmesh; FVmesh])
end

print_timer(AlphaStructures.to);


α = 0.0018122551435832204
α = 0.0029989337266015255
α = 0.005019266447699949
α = 0.006565148982631952
α = 0.006944322379707808
α = 0.007103889143013136
α = 0.009293169128121427
α = 0.009899721978901176
α = 0.010952280519803691
α = 0.1625143704715253
α = 687.5326012986985
 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:            89.3s / 4.54%           8.56GiB / 4.94%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 alphaFilter                1    4.00s  98.8%   4.00s    429MiB  99.1%   429MiB
   processuppersimplex  1.98k   93.1ms  2.30%  47.0μs   24.3MiB  5.63%  12.6KiB
     processlowsimplex  5.94k   57.1ms  1.41%  9.62μs   15.7MiB  3.62%  2.70KiB
       